In [ ]:
### Main header.

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [ ]:
### Parameters.

BATCH_SIZE_TRAIN = 64
BATCH_SIZE_TEST  = 1000
LEARNING_RATE    = 0.01
MOMENTUM         = 0.5
EPOCHS           = 10
SEED             = 1
LOG_INTERVAL     = 468
CON_CUDA         = False

torch.manual_seed(SEED)
kwargs = {"num_workers": 1, "pin_memory": True} if CON_CUDA else {}

In [ ]:
### Data loading.

# Train set: 60000 images
# Test set:  10000 images

train_data = torch.utils.data.DataLoader(
                 datasets.MNIST("../data/",
                                train = True,
                                download = True,
                                transform = transforms.Compose([
                                                transforms.ToTensor(),
                                                transforms.Normalize((0.1307,),(0.3081,))
                                            ])),
                 batch_size = BATCH_SIZE_TRAIN,
                 shuffle = True,
                 **kwargs
             )

test_data  = torch.utils.data.DataLoader(
                 datasets.MNIST("../data/",
                                train = False,
                                download = True,
                                transform = transforms.Compose([
                                                transforms.ToTensor(),
                                                transforms.Normalize((0.1307,),(0.3081,))
                                            ])),
                 batch_size = BATCH_SIZE_TEST,
                 shuffle = True,
                 **kwargs
             )

In [ ]:
### Model definition.

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels  = 1,
                               out_channels = 10,
                               kernel_size  = 5)
        
        self.conv2 = nn.Conv2d(in_channels  = 10,
                               out_channels = 20,
                               kernel_size  = 5)
        
        self.conv2_drop = nn.Dropout2d()
        
        self.fc1 = nn.Linear(in_features  = 320,
                             out_features = 50)
        
        self.fc2 = nn.Linear(in_features  = 50,
                             out_features = 10)
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320) # ???????????????????????????????
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training) # ?????????????????????????
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [ ]:
### Training and Testing definitions.

model = Net()

if CON_CUDA:
    model.cuda()

optimizer = optim.SGD(model.parameters(),
                      lr = LEARNING_RATE,
                      momentum = MOMENTUM)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_data):
        if CON_CUDA:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % LOG_INTERVAL == 0:
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(epoch,
                                                                           batch_idx * len(data),
                                                                           len(train_data.dataset),
                                                                           100. * batch_idx / len(train_data),
                                                                           loss.data[0]))

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_data:
        if CON_CUDA:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).data[0]
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(test_data.dataset)
    print("\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n".format(test_loss,
                                                                                 correct,
                                                                                 len(test_data.dataset),
                                                                                 100. * correct / len(test_data.dataset)))

In [ ]:
### Training and testing.

for epoch in range(1, EPOCHS + 1):
    train(epoch)
    test()

In [ ]:
### Serialization.

torch.save(model.state_dict(), "./state.pkl")

In [ ]:
### Deserialization and testing.

new_model = Net()
new_model.load_state_dict(torch.load("./state.pkl"))

def new_test():
    new_model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_data:
        if CON_CUDA:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = new_model(data)
        test_loss += F.nll_loss(output, target, size_average=False).data[0]
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(test_data.dataset)
    print("\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n".format(test_loss,
                                                                                 correct,
                                                                                 len(test_data.dataset),
                                                                                 100. * correct / len(test_data.dataset)))

new_test()